In [10]:
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)

In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# ROOT_FOLDER = "/content/drive/My Drive/513_Project"
ROOT_FOLDER = "NYPL-menus"# "NYPL-menus-cleaned"

OUTPUT_FOLDER = "NYPL-menus-cleaned" # "/content/drive/My Drive/513_Project_Clean"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

OUTPUT_FILE = []

MENU = 1
MENU_PAGE = 3
MENU_ITEM = 2
DISH = 0

# Data Cleaning Workflow Instructions

## Accessing Datasets

Each dataset can be accessed from the dataset collection using the following keys:

```python
dataset["MENU"]          # Menu data
dataset["MENU_PAGE"]     # Menu page data
dataset["MENU_ITEM"]     # Menu item data
dataset["DISH"]          # Dish data
```

## Cleaning the Cleaned Dataset

Update the ROOT_FOLDER variable to point to your cleaned data directory:

```python
ROOT_FOLDER = "NYPL-menus-cleaned"  # Path to cleaned data
```

In [12]:
dataset = []
OUTPUT_FILE = []

print(f"Loading datasets from {ROOT_FOLDER}...")
for filename in sorted(Path(ROOT_FOLDER).iterdir()):
    if filename.name.endswith(".csv"):
        print(f"Loading {filename.name}...")
        OUTPUT_FILE.append(filename.name.replace(".csv", "_fixed.csv").replace("_fixed_fixed", "_fixed"))
    if filename.name.endswith(".csv"):
        dataset.append(pd.read_csv(filename, na_values=[""]))
print("Datasets loaded.\n")

print("output files:")
print(OUTPUT_FILE)

Loading datasets from NYPL-menus...
Loading Dish.csv...
Loading Menu.csv...
Loading MenuItem.csv...
Loading MenuPage.csv...
Datasets loaded.

output files:
['Dish_fixed.csv', 'Menu_fixed.csv', 'MenuItem_fixed.csv', 'MenuPage_fixed.csv']


In [13]:
# IC 2: Date Outliner in Menu

# Extract first 4 digits of call_number (if not null)
dataset[MENU]["call_prefix"] = dataset[MENU]["call_number"].str[:4]

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4]

# Check constraint: call_prefix == date_year when both exist
ic2_violations = dataset[MENU][
    dataset[MENU]["call_number"].notna() &
    dataset[MENU]["date"].notna() &
    dataset[MENU]["call_prefix"].str.isnumeric() &
    (dataset[MENU]["date"].str.match(r"[^1]+") |
    dataset[MENU]["date"].str.match(r"[1][^89]+")) &
    (dataset[MENU]["call_prefix"] != dataset[MENU]["date_prefix"])
]

print(f"Before Cleaning Applied: {len(ic2_violations)}")
dataset[MENU].loc[ic2_violations.index][["id", "call_number", "date", "call_prefix", "date_prefix"]]

Before Cleaning Applied: 5


,id,call_number,date,call_prefix,date_prefix
525,13112,1900-2328,0190-03-06,1900,0190
6195,22951,1901-213,1091-01-27,1901,1091
14659,32265,1918-0387_wotm,2928-03-26,1918,2928
16917,34727,1912-0667_wotm,0001-01-01,1912,0001
16918,34728,1912-0668_wotm,0001-01-01,1912,0001


In [14]:

# IC 2: Date Outliner in Menu Cleaning
dataset[MENU].loc[ic2_violations.index, "date"] = (
    dataset[MENU].loc[ic2_violations.index, "call_prefix"] +
    dataset[MENU].loc[ic2_violations.index, "date"].str[4:]
)

print(f"After Cleaning Applied: {len(ic2_violations)}")
dataset[MENU].loc[ic2_violations.index][["id", "call_number", "date", "call_prefix", "date_prefix"]]

After Cleaning Applied: 5


,id,call_number,date,call_prefix,date_prefix
525,13112,1900-2328,1900-03-06,1900,0190
6195,22951,1901-213,1901-01-27,1901,1091
14659,32265,1918-0387_wotm,1918-03-26,1918,2928
16917,34727,1912-0667_wotm,1912-01-01,1912,0001
16918,34728,1912-0668_wotm,1912-01-01,1912,0001


In [15]:
# IC 3: Date Blank with call number year in Menu

# Extract first 4 digits of call_number (if not null)
dataset[MENU]["call_prefix"] = dataset[MENU]["call_number"].str[:4]

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4]

# Check constraint: date should not be blank when call_prefix is numeric
ic3_violations = dataset[MENU][
    dataset[MENU]["call_number"].notna() &
    dataset[MENU]["date"].isna() &
    dataset[MENU]["call_prefix"].str.isnumeric()
]

print(f"Violations found: {len(ic3_violations)}")
ic3_violations[["id", "call_number", "date", "call_prefix", "date_prefix"]].head(20)

Violations found: 47


,id,call_number,date,call_prefix,date_prefix
465,13042,1900-2517,NaN,1900,NaN
4598,20978,1906-783,NaN,1906,NaN
5025,21467,1886-036,NaN,1886,NaN
5400,21969,1887-028,NaN,1887,NaN
8633,25998,1900-189,NaN,1900,NaN
8732,26119,1899-606,NaN,1899,NaN
10093,27576,1910-881,NaN,1910,NaN
10426,27912,1973-0020_wotm,NaN,1973,NaN
10526,28012,1977-0004_wotm,NaN,1977,NaN
10560,28062,1978-0021_wotm,NaN,1978,NaN


In [16]:
# IC 3: Date Blank with call number year in Menu Cleaning
dataset[MENU].loc[ic3_violations.index, "date"] = (
    dataset[MENU].loc[ic3_violations.index, "call_prefix"] + "-01-01"
)

print(f"After Cleaning Applied: {len(ic3_violations)}")
dataset[MENU].loc[ic3_violations.index][["id", "call_number", "date", "call_prefix", "date_prefix"]]

After Cleaning Applied: 47


,id,call_number,date,call_prefix,date_prefix
465,13042,1900-2517,1900-01-01,1900,NaN
4598,20978,1906-783,1906-01-01,1906,NaN
5025,21467,1886-036,1886-01-01,1886,NaN
5400,21969,1887-028,1887-01-01,1887,NaN
8633,25998,1900-189,1900-01-01,1900,NaN
8732,26119,1899-606,1899-01-01,1899,NaN
10093,27576,1910-881,1910-01-01,1910,NaN
10426,27912,1973-0020_wotm,1973-01-01,1973,NaN
10526,28012,1977-0004_wotm,1977-01-01,1977,NaN
10560,28062,1978-0021_wotm,1978-01-01,1978,NaN


In [17]:
# IC 4: Date Blank with no date info in call number in Menu

# Extract first 4 digits of call_number (if not null)
dataset[MENU]["call_prefix"] = dataset[MENU]["call_number"].str[:4]

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4]

# Check constraint: date should not be blank when call_prefix is numeric
ic4_violations = dataset[MENU][
    dataset[MENU]["call_number"].notna() &
    dataset[MENU]["date"].isna() & (
    (dataset[MENU]["call_prefix"] == 'Zand') |
    (dataset[MENU]["call_prefix"] == 'Soet') |
    (dataset[MENU]["call_prefix"] == 'soet') |
    (dataset[MENU]["call_prefix"] == 'Bara') |
    (dataset[MENU]["call_prefix"] == '_wot'))
]

print(f"Violations found: {len(ic4_violations)}")
dataset[MENU].loc[ic4_violations.index][["id", "call_number", "date", "call_prefix", "date_prefix"]].head(20)

Violations found: 536


,id,call_number,date,call_prefix,date_prefix
8913,26347,Zander 11,NaN,Zand,NaN
8915,26349,Zander 13,NaN,Zand,NaN
8918,26352,Zander 16,NaN,Zand,NaN
8921,26355,Zander 19 undated,NaN,Zand,NaN
8922,26356,Zander 21 undated,NaN,Zand,NaN
8924,26358,Zander 23 undated,NaN,Zand,NaN
8925,26360,Zander 24 undated,NaN,Zand,NaN
8926,26361,Zander 25 undated,NaN,Zand,NaN
8930,26366,Zander 30 undated,NaN,Zand,NaN
8932,26368,Zander 32 undated,NaN,Zand,NaN


In [18]:

# IC 4: Date Blank with no date info in call number in Menu Cleaning
dataset[MENU]["date"] = dataset[MENU]["date"].ffill()
# .interpolate(method='nearest')

print(f"After Cleaning Applied: {len(ic4_violations)}")
dataset[MENU].loc[ic4_violations.index][["id", "call_number", "date", "call_prefix", "date_prefix"]]

After Cleaning Applied: 536


,id,call_number,date,call_prefix,date_prefix
8913,26347,Zander 11,1989-01-01,Zand,NaN
8915,26349,Zander 13,1969-12-01,Zand,NaN
8918,26352,Zander 16,1952-01-01,Zand,NaN
8921,26355,Zander 19 undated,1963-07-29,Zand,NaN
8922,26356,Zander 21 undated,1963-07-29,Zand,NaN
...,...,...,...,...,...
10224,27708,Soete 235A-B,1919-05-12,Soet,NaN
10225,27709,Soete 237A-C,1919-05-12,Soet,NaN
10226,27710,Soete 209A,1919-05-12,Soet,NaN
10228,27712,soete 56,1931-11-01,soet,NaN


In [19]:
# # IC 5: Date range outsite of 1890-1970 in Menu
# ic5_violations = dataset[MENU][
#     dataset[MENU]["date"].notna() &
#     (dataset[MENU]["date"].str[:4].astype(int) < 1890) |
#     (dataset[MENU]["date"].str[:4].astype(int) > 1970)
# ]["id"]

# print(f"Violations found: {len(ic5_violations)}")
# print(f"Menu dataset size: {len(dataset[MENU])}")
# print(f"MenuPage dataset size: {len(dataset[MENU_PAGE])}")
# print(f"MenuItem dataset size: {len(dataset[MENU_ITEM])}")
# dataset[MENU].loc[ic5_violations.index]

Violations found: 1248<br>
Menu dataset size: 17545<br>
MenuPage dataset size: 66937<br>
MenuItem dataset size: 1332726<br>

In [20]:
# # IC 5: Date range outsite of 1890-1970 in Menu Cleaning
# # dataset[MENU] = dataset[MENU][
# #     ~dataset[MENU].isin(ic5_violations)
# # ]
# dataset[MENU] = dataset[MENU][~dataset[MENU]['id'].isin(ic5_violations)]

# # Cant remove yet, due to unlinked items
# # dataset[MENU_PAGE] = dataset[MENU_PAGE][~dataset[MENU_PAGE]['menu_id'].isin(ic5_violations)]
# # dataset[MENU_ITEM] = dataset[MENU_ITEM][~dataset[MENU_ITEM]['menu_page_id'].isin(dataset[MENU_PAGE]['id'])]

# print(f"After Cleaning Applied: {len(ic5_violations)}")
# print(f"Menu dataset size: {len(dataset[MENU])}")
# print(f"MenuPage dataset size: {len(dataset[MENU_PAGE])}")
# print(f"MenuItem dataset size: {len(dataset[MENU_ITEM])}")

In [21]:
# # IC 5: Clean up (remove dishes that no longer appear in any menu items)
# print(f"Dish dataset size: {len(dataset[DISH])}")

# print("Cleaning up Dish dataset...")
# active_dish_ids = dataset[MENU_ITEM]['dish_id'].unique()
# dataset[DISH] = dataset[DISH][
#     dataset[DISH]['id'].isin(active_dish_ids)
# ]

# print(f"Dish dataset size: {len(dataset[DISH])}")

In [22]:
# IC 6: Date blank in Dish with Dependency on Menu

# Extract first 4 digits of date (if not null)
dataset[MENU]["date_prefix"] = dataset[MENU]["date"].str[:4].astype('int64')

dish_appearances = (
    dataset[MENU_ITEM][['id', 'dish_id', 'menu_page_id']]
    .merge(dataset[MENU_PAGE][['id', 'menu_id']],
            left_on='menu_page_id', right_on='id', suffixes=('', '_page'))
    .merge(dataset[MENU], left_on='menu_id', right_on='id')
    .groupby('dish_id')['date_prefix']
    .agg(['min', 'max'])
    .reset_index()
    .rename(columns={'min': 'calc_first', 'max': 'calc_last'})
    .astype({'calc_first': 'int64', 'calc_last': 'int64'})
)

dishes = dataset[DISH].merge(
    dish_appearances, left_on='id', right_on='dish_id', how='left')

ic6_violations_6_first = dishes[
    (dishes['calc_first'].notna() &(
    (dishes['first_appeared'].isna()) |
    (dishes['first_appeared'] == 0) |
    (dishes['first_appeared'] == 1) |
    (dishes['calc_first'].notna() & (dishes['first_appeared'] > dishes['calc_first']))))
]

ic6_violations_6_last = dishes[
    (dishes['calc_last'].notna()) & (
    (dishes['last_appeared'].isna()) |
    (dishes['last_appeared'] == 0) |
    (dishes['last_appeared'] == 2928) |
    (dishes['calc_last'].notna() & (dishes['last_appeared'] < dishes['calc_last'])))
]

print(f"Violations found: {len(ic6_violations_6_first) + len(ic6_violations_6_last)}")
dishes.loc[ic6_violations_6_first.index ][
    ["id", "name", "first_appeared", "last_appeared", "calc_first", "calc_last"]
]

Violations found: 110910


,id,name,first_appeared,last_appeared,calc_first,calc_last
13,15,Celery,1,2928,1852.0,2012.0
24,26,Clams,1881,1970,1859.0,1970.0
25,27,Oysters,1862,1963,1859.0,1990.0
34,38,Apple Sauce,1,1987,1856.0,1987.0
72,78,Vegetable,1892,1987,1889.0,1987.0
...,...,...,...,...,...,...
423392,515673,Boiled: Corned beef & cabbage,0,0,1882.0,1882.0
423393,515674,Boiled: Knuckle of Veal & Bacon,0,0,1882.0,1882.0
423394,515675,Roast: Turkey & Cranberry Sauce,0,0,1882.0,1882.0
423395,515676,"Claret: Chateau Larose, Cruse et Fils Freres",0,0,1883.0,1883.0


In [23]:
# IC 6: Date blank in Dish with Dependency on Menu Cleaning
dataset[DISH].loc[ic6_violations_6_first.index, 'first_appeared'] = dishes.loc[ic6_violations_6_first.index, 'calc_first']

dataset[DISH].loc[ic6_violations_6_last.index, 'last_appeared'] = dishes.loc[ic6_violations_6_last.index, 'calc_last']

dataset[DISH]['first_appeared'] = dataset[DISH]['first_appeared'].ffill()
dataset[DISH]['last_appeared'] = dataset[DISH]['last_appeared'].ffill()

print(f"After Cleaning Applied: {len(ic6_violations_6_first)}")
dataset[DISH].loc[ic6_violations_6_first.index][['id', 'first_appeared', 'last_appeared']]

After Cleaning Applied: 54311


,id,first_appeared,last_appeared
13,15,1852,2012
24,26,1859,1970
25,27,1859,1990
34,38,1856,1987
72,78,1889,1987
...,...,...,...
423392,515673,1882,1882
423393,515674,1882,1882
423394,515675,1882,1882
423395,515676,1883,1883


In [26]:
# IC 7: Date Zero in Dish with no dependencies on Menu
ic7_violations_1 = dataset[DISH][
    (dataset[DISH]["first_appeared"] == 0) |
    (dataset[DISH]["last_appeared"] == 0)
]

ic7_violations_2 = dataset[MENU_ITEM][
    dataset[MENU_ITEM]["dish_id"].isin(dataset[DISH].loc[ic7_violations_1.index, "id"])
]

ic7_violations_3 = dataset[MENU_PAGE][
    dataset[MENU_PAGE]["id"].isin(ic7_violations_2["menu_page_id"])
]

ic7_violations_4 = dataset[MENU][
    dataset[MENU]["id"].isin(ic7_violations_3["menu_id"])
]

ic7_violations_first = dataset[DISH][(dataset[DISH]['first_appeared'] == 0) | (dataset[DISH]['first_appeared'] == 1)]

ic7_violations_last = dataset[DISH][dataset[DISH]['last_appeared'] == 0]

print(f"Violations found: {len(ic7_violations_1)}")
dataset[DISH].loc[ic7_violations_1.index][['id', 'first_appeared', 'last_appeared']]

# dataset[MENU_ITEM].loc[ic7_violations_2.index]
# dataset[MENU_PAGE].loc[ic7_violations_3.index]
# dataset[MENU].loc[ic7_violations_4.index]


Violations found: 4805


,id,first_appeared,last_appeared
70131,88446,0,1969
106416,132992,0,0
110907,138404,0,0
110908,138405,0,0
110909,138406,0,0
...,...,...,...
423321,515602,0,0
423323,515604,0,0
423339,515620,0,0
423365,515646,0,0


In [ ]:
# IC 7: Date Zero in Dish with no dependencies on Menu Cleaning
dataset[DISH].loc[ic7_violations_first.index, 'first_appeared'] = dataset[DISH].loc[ic7_violations_first.index, 'first_appeared'].replace(0, np.nan)
dataset[DISH].loc[ic7_violations_last.index, 'last_appeared'] = dataset[DISH].loc[ic7_violations_last.index, 'last_appeared'].replace(0, np.nan)

dataset[DISH]['first_appeared'] = dataset[DISH]['first_appeared'].ffill()
dataset[DISH]['last_appeared'] = dataset[DISH]['last_appeared'].ffill()

print(f"After Cleaning Applied: {len(ic7_violations_1)}")
dataset[DISH].loc[ic7_violations_1.index][['id', 'first_appeared', 'last_appeared']]

After Cleaning Applied: 4805


,id,first_appeared,last_appeared
70131,88446,1906.0,1969.0
106416,132992,1937.0,1971.0
110907,138404,1973.0,1973.0
110908,138405,1973.0,1973.0
110909,138406,1973.0,1973.0
...,...,...,...
423321,515602,1953.0,1953.0
423323,515604,1953.0,1953.0
423339,515620,1910.0,1910.0
423365,515646,1965.0,1965.0


In [ ]:
# IC 8: Temporal consistency in Dish
ic8_violations = dataset[DISH][dataset[DISH]["first_appeared"] > dataset[DISH]["last_appeared"]]

print(f"Violations found: {len(ic8_violations)}")
ic8_violations.head(10)

Violations found: 3


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
197050,250693,SURI LEBERLI - Shredded Calf's Liver Flambe in...,NaN,0,1,1945.0,1901.0,NaN,NaN
197053,250699,"SWISS MINCED VEAL, ROESTI",NaN,0,1,1945.0,1901.0,NaN,NaN
241858,306637,CARRE DE PORC FROID,NaN,0,1,1945.0,1944.0,0.0,0.0


In [ ]:
# IC 8: Temporal consistency in Dish cleaning
dataset[DISH].loc[ic8_violations.index, "last_appeared"] = dataset[DISH].loc[ic8_violations.index, "first_appeared"]

print(f"After Cleaning Applied: {len(ic8_violations)}")
dataset[DISH].loc[ic8_violations.index]

After Cleaning Applied: 3


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price
197050,250693,SURI LEBERLI - Shredded Calf's Liver Flambe in...,NaN,0,1,1945.0,1945.0,NaN,NaN
197053,250699,"SWISS MINCED VEAL, ROESTI",NaN,0,1,1945.0,1945.0,NaN,NaN
241858,306637,CARRE DE PORC FROID,NaN,0,1,1945.0,1945.0,0.0,0.0


In [ ]:
# IC 9: Date range outsite of 1880-2000 in Dish
ic9_violations = dataset[DISH][
    dataset[DISH]["first_appeared"].notna() &
    dataset[DISH]["last_appeared"].notna() & (
    (dataset[DISH]["first_appeared"] > 2000) |
    (dataset[DISH]["last_appeared"] < 1880))
]
print(f"Violations found: {len(ic9_violations)}")
dataset[DISH].loc[ic9_violations.index][["id", "name", "first_appeared", "last_appeared"]].head(10)

Violations found: 7065


,id,name,first_appeared,last_appeared
2173,2605,SAUTERNES.,1865.0,1865.0
3252,4055,"Soups without meats,",1865.0,1865.0
3269,4078,Bropiled Blue Fish,1865.0,1865.0
3273,4082,"Broiled Cusk, Cream Sauce",1865.0,1865.0
3373,4203,Stewed Eels,1858.0,1865.0
3401,4239,"Baked Cod, Port Sauce",1858.0,1865.0
3424,4265,"Chicken and Pork, White Sauce",1865.0,1865.0
3427,4269,Shoulder Corned Pork,1865.0,1865.0
3459,4308,Stewed Calf's Liver,1865.0,1865.0
3460,4309,"Loin of Veal, Stuffed",1865.0,1865.0


In [ ]:
# IC 9: Date range outsite of 1880-2000 in Dish Cleaning
dataset[DISH] = dataset[DISH][~dataset[DISH]['id'].isin(ic9_violations['id'])]
print(f"After Cleaning Applied: {len(ic9_violations)}")

print(dataset[DISH]['first_appeared'].agg(['min', 'max']))

print(dataset[DISH]['last_appeared'].agg(['min', 'max']))

After Cleaning Applied: 7065
min    1851.0
max    2000.0
Name: first_appeared, dtype: float64
min    1880.0
max    2015.0
Name: last_appeared, dtype: float64


In [ ]:
# IC 10: Blank lowest_price and highest_price in Dish
ic10_violations_1 = dataset[DISH][
    dataset[DISH]["lowest_price"].isna() &
    dataset[DISH]["highest_price"].isna()
]
print(f"Violations found: {len(ic10_violations_1)}")
dataset[DISH].loc[ic10_violations_1.index][["id", "name", "lowest_price", "highest_price"]].head(10)

Violations found: 28665


,id,name,lowest_price,highest_price
32,34,Russian Caviare on Toast,NaN,NaN
35,39,Potage a la Victoria,NaN,NaN
55,60,Hafergrutze,NaN,NaN
58,63,Apfelsinen,NaN,NaN
60,65,Milchreis,NaN,NaN
79,87,Hot or cold ribs of beef,NaN,NaN
127,135,Consomme aux Quenelle's,NaN,NaN
128,136,Milk rice,NaN,NaN
161,170,Baked Stuffed Mullet & Sauce Pomard,NaN,NaN
293,346,Grilled Mutton Chops,NaN,NaN


In [ ]:
# IC 10: Blank lowest_price and highest_price in Dish with Dependency on Menu Item
menu_item_price = (
    dataset[MENU_ITEM][['id', 'dish_id', 'price']]
    .groupby('dish_id')['price']
    .agg(['min', 'max'])
    .reset_index()
    .rename(columns={'min': 'calc_lowest', 'max': 'calc_highest'})
)

dataset[DISH] = dataset[DISH].merge(
    menu_item_price, left_on='id', right_on='dish_id', how='left')

ic10_violations_2 = dataset[DISH][
    (dataset[DISH]["lowest_price"].isna() | dataset[DISH]["lowest_price"].isna()) &
    (dataset[DISH]["calc_lowest"].notna() | dataset[DISH]["calc_highest"].notna())
]

print(f"Violations found: {len(ic10_violations_2)}")
dataset[DISH].loc[ic10_violations_2.index]

Violations found: 116


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price,dish_id,calc_lowest,calc_highest
17098,21809,Aepfel,NaN,78,78,1899.0,1912.0,NaN,NaN,21809.0,1.00,1.00
20110,25612,Green Gage Pie,NaN,17,17,1896.0,1916.0,NaN,NaN,25612.0,0.25,0.25
21477,27359,Salami Sausage,NaN,14,15,1900.0,1970.0,NaN,NaN,27359.0,0.45,0.45
25716,32795,Bass Ale on Draught,NaN,2,2,1900.0,1913.0,NaN,NaN,32795.0,0.10,0.10
28019,35844,French Fried Potatoes.,NaN,2,2,1900.0,1914.0,NaN,NaN,35844.0,0.10,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...
381987,475769,Fresh Cream,NaN,2,2,1964.0,1964.0,NaN,NaN,475769.0,2.00,2.00
382543,476680,Green Goddess,NaN,2,2,1971.0,1975.0,NaN,NaN,476680.0,0.75,0.75
395104,491556,Crème à la Reine,NaN,2,2,1891.0,1972.0,NaN,NaN,491556.0,5.25,5.25
396293,492912,Sauté potatoes,NaN,2,2,1900.0,1913.0,NaN,NaN,492912.0,0.25,0.25


In [ ]:
# IC 10: Blank lowest_price and highest_price in Dish with No price info in Menu Item
ic10_violations_3 = dataset[DISH][
    dataset[DISH]["lowest_price"].isna() &
    dataset[DISH]["highest_price"].isna() &
    dataset[DISH]["calc_lowest"].isna() &
    dataset[DISH]["calc_highest"].isna()
]

print(f"Violations found: {len(ic10_violations_3)}")
dataset[DISH].loc[ic10_violations_3.index]

Violations found: 28549


,id,name,description,menus_appeared,times_appeared,first_appeared,last_appeared,lowest_price,highest_price,dish_id,calc_lowest,calc_highest
32,34,Russian Caviare on Toast,NaN,3,3,1900.0,1900.0,NaN,NaN,34.0,NaN,NaN
35,39,Potage a la Victoria,NaN,5,5,1899.0,1901.0,NaN,NaN,39.0,NaN,NaN
55,60,Hafergrutze,NaN,205,218,1899.0,1910.0,NaN,NaN,60.0,NaN,NaN
58,63,Apfelsinen,NaN,181,184,1899.0,1935.0,NaN,NaN,63.0,NaN,NaN
60,65,Milchreis,NaN,135,135,1899.0,1910.0,NaN,NaN,65.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
414251,513487,Beurre noir,NaN,1,2,1899.0,1899.0,NaN,NaN,513487.0,NaN,NaN
414252,513488,North-sea-Turbot,NaN,1,1,1899.0,1899.0,NaN,NaN,513488.0,NaN,NaN
414253,513489,Caulyflower au gratin,NaN,1,1,1899.0,1899.0,NaN,NaN,513489.0,NaN,NaN
414254,513490,Veal-tongue,NaN,1,1,1899.0,1899.0,NaN,NaN,513490.0,NaN,NaN


In [ ]:
# IC 10: Blank lowest_price and highest_price in Dish with Dependency on Menu Item Cleaning
dataset[DISH].loc[ic10_violations_2.index, "lowest_price"] = dataset[DISH].loc[ic10_violations_2.index, "calc_lowest"]
dataset[DISH].loc[ic10_violations_2.index, "highest_price"] = dataset[DISH].loc[ic10_violations_2.index, "calc_highest"]

print(f"After Cleaning Applied: {len(ic10_violations_2)}")
dataset[DISH].loc[ic10_violations_2.index][["id", "name", "lowest_price", "highest_price"]].head(10)

After Cleaning Applied: 116


,id,name,lowest_price,highest_price
17098,21809,Aepfel,1.00,1.00
20110,25612,Green Gage Pie,0.25,0.25
21477,27359,Salami Sausage,0.45,0.45
25716,32795,Bass Ale on Draught,0.10,0.10
28019,35844,French Fried Potatoes.,0.10,0.10
36558,47246,Blueberry Pies,0.25,0.25
47392,61246,Saucission de Lyon,0.30,0.30
61521,78990,Chas. Heidsieck,2.00,2.00
65053,83269,New String Beans.,0.40,0.40
155720,200117,Kartoffel-Purée,0.30,0.30


In [ ]:
# IC 10: Blank lowest_price and highest_price in Dish with No price info in Menu Item Cleaning
print(f"Before Cleaning Applied Dish dataset size: {len(dataset[DISH])}")
dataset[DISH] = dataset[DISH].drop(ic10_violations_3.index, errors='ignore')

print(f"After Cleaning Applied: {len(ic10_violations_3)}")
print(f"Dish dataset size: {len(dataset[DISH])}")

Before Cleaning Applied Dish dataset size: 416332
After Cleaning Applied: 28549
Dish dataset size: 387783


In [ ]:
# IC 11: Missing or Null Place Values in Menu
ic11_violations = dataset[MENU][
    dataset[MENU]["place"].isna() |
    (dataset[MENU]["place"].str.strip() == "")
]

print(f"IC 11 Violations (missing/blank place): {len(ic11_violations)}")
dataset[MENU].loc[ic11_violations.index][["id", "place"]].head()

IC 11 Violations (missing/blank place): 9422


,id,place
15,12478,NaN
58,12526,NaN
102,12583,NaN
103,12584,NaN
104,12585,NaN


In [ ]:
# Fix for IC 11: Replace missing/blank 'place' with "Unknown"
dataset[MENU]["place"] = dataset[MENU]["place"].fillna("Unknown")
dataset[MENU]["place"] = dataset[MENU]["place"].apply(lambda x: "Unknown" if str(x).strip() == "" else x)

ic11_violations_after = dataset[MENU][
    dataset[MENU]["place"].isna() |
    (dataset[MENU]["place"].str.strip() == "")
]

print(f"IC 11 Violations After Cleaning: {len(ic11_violations_after)}")
dataset[MENU].loc[ic11_violations.index][["id", "place"]].head()

IC 11 Violations After Cleaning: 0


,id,place
15,12478,Unknown
58,12526,Unknown
102,12583,Unknown
103,12584,Unknown
104,12585,Unknown


In [ ]:
# IC 12: Place values that are overly specific or not cleanly grouped
valid_groups = [
    "United States", "Italy", "France", "Canada", "England", "Japan", "Germany",
    "China", "Austria", "Bahamas", "Hungary", "Cuba", "Switzerland",
    "Shipboard", "Trainboard", "Unknown", "Other"
]

ic12_violations = dataset[MENU][
    ~dataset[MENU]["place"].isin(valid_groups)
]

print(f"IC 12 Violations (uncategorized place values): {len(ic12_violations)}")
dataset[MENU].loc[ic12_violations.index][["id", "place"]].head()

IC 12 Violations (uncategorized place values): 8111


,id,place
0,12463,"HOT SPRINGS, AR"
1,12464,"MILWAUKEE, [WI];"
2,12465,DAMPFER KAISER WILHELM DER GROSSE;
3,12466,DAMPFER KAISER WILHELM DER GROSSE;
4,12467,DAMPFER KAISER WILHELM DER GROSSE;


In [ ]:
import re

def categorize_place(place):
    if pd.isnull(place):
        return "Unknown"

    place = str(place).upper().strip()
    place = re.sub(r"[\[\];]", "", place)

    # 1. Unknown or ambiguous
    if place in ["", "UNKNOWN", "?", "Unknown"]:
        return "Unknown"

    # 2. Match by U.S. state abbreviation
    us_states = [
        "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", "HI", "IA", "ID", "IL", "IN",
        "KS", "KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO", "MS", "MT", "NC", "ND", "NE", "NH", "NJ",
        "NM", "NV", "NY", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA",
        "WI", "WV", "WY", "FLA.", "N.C.", "S.C.", "R.I.", "MASS", "OHIO", "PA.", "TEXAS", "TENN.", "MICH"
    ]
    if any(f", {state}" in place or place.endswith(f" {state}") or place.endswith(f"{state}") for state in us_states):
        return "United States"

    # 3. Match common U.S. cities
    us_city_terms = ["NEW YORK", "NYC", "ST.AUGUSTINE", "TAMPA", "CINCINNATI", "LOS ANGELES", "CHICAGO", "[NY]", "NY", "D.C", "SAN FRANCISCO"]
    if any(city in place for city in us_city_terms):
        return "United States"

    # 4. Street addresses (NYC indicator)
    if any(term in place for term in [
        "ST.", "STREET", "AVENUE", "MADISON", "COLUMBUS", "PARK AVE", "LEXINGTON", "5TH AVE", "BROADWAY", "Park"
    ]):
        return "United States"

    # 5. Known U.S. landmarks (NY hotels/restaurants)
    if any(name in place for name in ["WALDORF", "DELMONICO", "PLAZA", "GRAMERCY", "SONOMA", "VENTURA"]):
        return "United States"

    # 6. Shipboard patterns
    if any(term in place for term in [
        "R.M.S.", "RMS", "S.S.", "SS ", "STEAMER", "DAMPFER", "AN BORD DER", "ON BOARD", "EN ROUTE", "AT SEA",
        "SCHNELLDAMPFER", "KAISER", "KONIGIN", "ROUTES", "ABOARD", "STEAMSHIP", "ROUTE", "SS", "SEA", "USMS"
    ]):
        return "Shipboard"

    # 7. Trainboard patterns
    if any(term in place for term in ["DINING CAR", "SOUTHERN PACIFIC", "PULLMAN", "CAR"]):
        return "Trainboard"

    # 8. Country-level terms
    if "FRANCE" in place or "PARIS" in place:
        return "France"
    if "GERMANY" in place or "HAMBURG" in place or "BREMEN" in place:
        return "Germany"
    if "CANADA" in place or "ONTARIO" in place or "TORONTO" in place:
        return "Canada"
    if "ENGLAND" in place or "LONDON" in place:
        return "England"
    if "ITALY" in place or "ROME" in place or "FLORENCE" in place:
        return "Italy"
    if "JAPAN" in place or "TOKYO" in place or "YOKOHAMA" in place:
        return "Japan"
    if "CHINA" in place or "HONG KONG" in place or "SHANGHAI" in place:
        return "China"
    if "BAHAMAS" in place or "BERMUDA" in place:
        return "Bahamas"
    if "HAVANA" in place:
        return "Cuba"
    if "BUDAPEST" in place:
        return "Hungary"
    if "ZERMATT" in place or "SWITZERLAND" in place:
        return "Switzerland"
    if "VIENNA" in place or "AUSTRIA" in place:
        return "Austria"

    if any(name in place for name in ["Hotel", "HOTEL", "INN", "CLUB", "HOUSE", "Maison", "Lounge"]):
        return "Hotel (Unknown Country)"

    # 9. Catch-all
    return "Other"

In [ ]:
# Apply place categorization function
dataset[MENU]["cleaned_place"] = dataset[MENU]["place"].apply(categorize_place)
# Recheck IC 12 violations
ic12_violations_after = dataset[MENU][
    ~dataset[MENU]["cleaned_place"].isin(valid_groups)
]

print(f"IC 12 Violations After Cleaning: {len(ic12_violations_after)}")
dataset[MENU].loc[ic12_violations_after.index][["id", "place", "cleaned_place"]].head()

IC 12 Violations After Cleaning: 244


,id,place,cleaned_place
52,12519,"RED ROOM,BOHEMIAN CLUB",Hotel (Unknown Country)
205,12716,[THE KNICKERBOCKER CLUB?],Hotel (Unknown Country)
206,12717,SAINT LOUIS CLUB,Hotel (Unknown Country)
267,12802,ROUND TABLE CLUB,Hotel (Unknown Country)
335,12881,NARRAGANSETT HOTEL,Hotel (Unknown Country)


In [ ]:
# IC 17: Currency inconsistency in Menu

ic17_violations = dataset[MENU][
    dataset[MENU]["cleaned_place"].notna() & 
    dataset[MENU]["cleaned_place"].str.contains("United States") &
    (dataset[MENU]["currency"].isna() | (dataset[MENU]["currency"].notna() &
    (dataset[MENU]["currency"] != "Dollars")))
]

print(f"IC 17 Violations (currency inconsistency): {len(ic17_violations)}")

dataset[MENU].loc[ic17_violations.index][["id", "cleaned_place", "currency"]].head()

IC 17 Violations (currency inconsistency): 3675


,id,cleaned_place,currency
0,12463,United States,NaN
1,12464,United States,NaN
10,12473,United States,NaN
19,12482,United States,NaN
21,12484,United States,NaN


In [ ]:
# IC 17: Place consistency in Menu Cleaning
dataset[MENU].loc[ic17_violations.index, "currency"] = "Dollars"

print(f"After Cleaning Applied: {len(ic17_violations)}")

dataset[MENU].loc[ic17_violations.index][["id", "cleaned_place", "currency"]].head()

After Cleaning Applied: 3675


,id,cleaned_place,currency
0,12463,United States,Dollars
1,12464,United States,Dollars
10,12473,United States,Dollars
19,12482,United States,Dollars
21,12484,United States,Dollars


In [ ]:
# IC 1: find outliners in the price of dishes
# Filter Dollar menus and join with menu items
ic_1_1 = (
    dataset[MENU_ITEM][['id', 'price', 'menu_page_id']]
    .merge(dataset[MENU_PAGE][['id', 'menu_id']], 
            left_on='menu_page_id', right_on='id', how='left')
    .merge(dataset[MENU][['id', 'currency']], 
            left_on='menu_id', right_on='id', how='left')
)

ic_1_1 = ic_1_1[(ic_1_1['currency'] == 'Dollars')]

# Calculate percentiles
ic_1_2 = ic_1_1['price'].dropna()
percentiles = {
    'p90': np.percentile(ic_1_2, 90),
    'p95': np.percentile(ic_1_2, 95),
    'p999': np.percentile(ic_1_2, 99.9)
}

ic_1_3_1 = ic_1_1[
    (ic_1_1['price'] >= percentiles['p999']) |
    ic_1_1['price'].isna()
]

# Identify outliers (above 99.9 percentile) and empty prices
ic_1_3 = ic_1_1[
    (ic_1_1['price'] >= percentiles['p999']) |
    ic_1_1['price'].isna() |
    (ic_1_1['price'] <= 0)
]

print("Price Percentiles (Dollar items):")
print(f"90th: ${percentiles['p90']:.2f}")
print(f"95th: ${percentiles['p95']:.2f}") 
print(f"999th: ${percentiles['p999']:.2f}")

print(f"\nFound {len(ic_1_3)} potential outliers:")
print(f"\nFound {len(ic_1_3_1)} items with price >= 99.9 percentile:")
print(ic_1_3[['id', 'price', 'menu_id']].head(10))


Price Percentiles (Dollar items):
90th: $1.75
95th: $3.25
999th: $75.00

Found 71888 potential outliers:
          id  price  menu_id
12   12882.0    NaN    12882
203  12469.0    NaN    12469
204  12469.0   75.0    12469
205  13155.0    NaN    13155
279  13117.0    NaN    13117
280  13117.0    NaN    13117
281  13117.0    NaN    13117
282  13117.0    NaN    13117
283  13117.0    NaN    13117
284  13117.0    NaN    13117


In [ ]:
# IC 1: find outliners in the price of dishes Cleaning
print(f"Before Cleaning Applied: {len(dataset[MENU_ITEM])}")
len_before = len(dataset[MENU_ITEM])
dataset[MENU_ITEM] = dataset[MENU_ITEM].drop(ic_1_3.index, errors='ignore')
print(f"After Cleaning Applied: {len(dataset[MENU_ITEM])}")
print(f"Removed {len_before - len(dataset[MENU_ITEM])} items from MenuItem dataset due to price outliers.")

Before Cleaning Applied: 1332726
After Cleaning Applied: 1149827
Removed 182899 items from MenuItem dataset due to price outliers.


In [ ]:
# IC 18: the average price of dishes every 20 years from 1880 to 2000

# IC 18.1: Dollar currency in Menu
ic_18_1 = dataset[MENU][dataset[MENU]["currency"] == "Dollars"]
# IC 18.2: Connect dishes to their menu appearances
ic_18_2 = (
    dataset[MENU_ITEM][['dish_id', 'price', 'menu_page_id']]
    .merge(dataset[MENU_PAGE][['id', 'menu_id']], 
            left_on='menu_page_id', right_on='id')
    .merge(ic_18_1, left_on='menu_id', right_on='id')
    .merge(dataset[DISH][['id', 'first_appeared', 'last_appeared']],
            left_on='dish_id', right_on='id')
)

# IC 18.3:  Create 20-year periods
periods = [(year, year+19) for year in range(1880, 2000, 20)]
periods[5] = (1980, 2000)

# IC 18.4: Calculate average price for each period
avg_prices = []
for start, end in periods:
# IC 18.5: Find dishes active during each period
    IC_18_5 = ic_18_2[
        (ic_18_2['first_appeared'] <= end) & 
        (ic_18_2['last_appeared'] >= start)
    ]

    IC_18_5['price'].head()

    # Calculate all required statistics
    avg_price = IC_18_5['price'].mean()
    median_price = IC_18_5['price'].median()
    standard_deviation = IC_18_5['price'].std()
    max_price = IC_18_5['price'].max()
    min_price = IC_18_5['price'].min()    

    avg_prices.append({
        'period': f"{start}-{end}",
        'avg_price': round(avg_price, 2),
        'median_price': round(median_price, 2),
        'std_dev': round(standard_deviation, 2),
        'max_price': round(max_price, 2),
        'min_price': round(min_price, 2),
        'num_dishes': len(IC_18_5['dish_id'].unique()),
        'num_menus': len(IC_18_5['menu_id'].unique())
    })

print("IC 18: Average price of dishes every 20 years from 1880 to 2000")
# convert avg_prices to DataFrame
ic_18_df = pd.DataFrame(avg_prices)
ic_18_df.head(6)

IC 18: Average price of dishes every 20 years from 1880 to 2000


,period,avg_price,median_price,std_dev,max_price,min_price,num_dishes,num_menus
0,1880-1899,0.86,0.30,4.31,175.0,0.01,13586,5207
1,1900-1919,0.90,0.35,4.50,175.0,0.00,130055,5339
2,1920-1939,0.83,0.35,3.99,175.0,0.01,33660,5311
3,1940-1959,1.00,0.35,4.88,175.0,0.01,48309,5339
4,1960-1979,1.35,0.35,6.09,175.0,0.01,31545,5338
5,1980-2000,1.97,0.35,9.03,175.0,0.01,16870,5277


In [ ]:
# Export the cleaned dataset
dataset[MENU].drop(columns=["date_prefix", "call_prefix"], inplace=True)
dataset[DISH].drop(columns=["calc_lowest", "calc_highest"], inplace=True)

for i in range(len(dataset)):
    dataset[i].to_csv(Path(OUTPUT_FOLDER) / OUTPUT_FILE[i], index=False)

In [ ]:
# IC 13:  DINNER/LUNCH/TIFFIN/BREAKFAST in different language or descriptions 

ic13_violations = dataset[MENU][
    dataset[MENU]['occasion'].fillna('na').str.upper().str.contains(
    'FRUHSTUCK|MITTAGESSEN|ABENDESSEN|DINER|DEJEUNER|NOON|(\bTEA\b)|MIDDAY|EVENING', regex=True) |
    dataset[MENU]['event'].fillna('na').str.upper().str.contains(
    'FRUHSTUCK|MITTAGESSEN|ABENDESSEN|DINER|DEJEUNER|NOON|(\bTEA\b)|MIDDAY|EVENING', regex=True)
]

print(f"Violations found: {len(ic13_violations)}")
dataset[MENU].loc[ic13_violations.index][['event','occasion']]


Violations found: 286


C:\Users\Hwu55\AppData\Local\Temp\ipykernel_7948\741432422.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset[MENU]['occasion'].fillna('na').str.upper().str.contains(
C:\Users\Hwu55\AppData\Local\Temp\ipykernel_7948\741432422.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset[MENU]['event'].fillna('na').str.upper().str.contains(


,event,occasion
2,FRUHSTUCK/BREAKFAST;,NaN
7,FRUHSTUCK/BREAKFAST,NaN
203,EVENING DINNER,NaN
208,EVENING MEAL,NaN
415,DINNER,EVENING DINNER
...,...,...
9366,Afternoon tea,NaN
9379,Afternoon tea,NaN
9514,Midday fare,NaN
9576,afternoon tea,NaN


In [ ]:
## IC 13:  DINNER/LUNCH/TIFFIN/BREAKFAST in different language or descriptions. Cleaning.

# @BEGIN openrefine_basic_clean_cluster
# @PARAM OUTPUT_FOLDER
# @IN dataset[MENU]  @AS Menu_fixed  @URI file:{OUTPUT_FOLDER}/Menu_fixed.csv
# @OUT dataset[MENU]  @AS Menu_fixed_openrefined  @URI file:/Menu_fixed_ic13_ORCluster.csv
# @END openrefine_basic_clean_cluster

In [ ]:
input_filename = "Menu_fixed_ic13_ORCluster.csv"
dataset[MENU] = pd.read_csv( input_filename, na_values=[""] )

In [ ]:
# IC 13: cleaning results from openrefine

print(f"Before Cleaning Applied Dish dataset size: {dataset[MENU].shape}")

ic13_violations_after = dataset[MENU][
    dataset[MENU]['occasion'].fillna('na').str.upper().str.contains(
    'FRUHSTUCK|MITTAGESSEN|ABENDESSEN|DINER|DEJEUNER|NOON|(\bTEA\b)|MIDDAY|EVENING|MIDDAG', regex=True) |
    dataset[MENU]['event'].fillna('na').str.upper().str.contains(
    'FRUHSTUCK|MITTAGESSEN|ABENDESSEN|DINER|DEJEUNER|NOON|(\bTEA\b)|MIDDAY|EVENING|MIDDAG', regex=True)
]

dataset[MENU]['meal_type'] = ""
## Create a new column meal_type that isolates out type of meal
breakfast_filter = dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('BREAKFAST') | dataset[MENU]['event'].fillna('na').str.upper().str.contains('BREAKFAST')
dataset[MENU].loc[ breakfast_filter, 'meal_type'] = dataset[MENU].loc[ breakfast_filter, 'meal_type'] + "B"

lunch_filter = dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('LUNCH') | dataset[MENU]['event'].fillna('').str.upper().str.contains('LUNCH')
dataset[MENU].loc[ lunch_filter, 'meal_type'] = dataset[MENU].loc[ lunch_filter, 'meal_type'] + "L"

tiffin_filter = dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('TIFFIN') | dataset[MENU]['event'].fillna('').str.upper().str.contains('TIFFIN')
dataset[MENU].loc[ tiffin_filter, 'meal_type'] = dataset[MENU].loc[ tiffin_filter, 'meal_type'] + "T"

dinner_filter = dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('SUPPER|DINNER') | dataset[MENU]['event'].fillna('').str.upper().str.contains('SUPPER|DINNER')
dataset[MENU].loc[ dinner_filter, 'meal_type'] = dataset[MENU].loc[ dinner_filter, 'meal_type'] + "D"


print(f"IC 13 Violations After Cleaning: {len(ic13_violations_after)}")
print(f"Dish dataset size: {dataset[MENU].shape}")


C:\Users\Hwu55\AppData\Local\Temp\ipykernel_7948\280169130.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset[MENU]['occasion'].fillna('na').str.upper().str.contains(
C:\Users\Hwu55\AppData\Local\Temp\ipykernel_7948\280169130.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dataset[MENU]['event'].fillna('na').str.upper().str.contains(


Before Cleaning Applied Dish dataset size: (17545, 21)
IC 13 Violations After Cleaning: 0
Dish dataset size: (17545, 22)


In [ ]:
# IC 14: event and occasion field contains pure information on menu. 
# e.g WINE XXX LIST/ROOM SERVICE
# those are not helpful to identify what event there is 

ic14_violations = dataset[MENU][
    dataset[MENU]['event'].fillna('na').str.upper().str.contains('[A-z]+\sLIST',regex=True) | 
    dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('[A-z]+\sLIST',regex=True) |
    dataset[MENU]['event'].fillna('na').str.upper().str.contains('^ROOM SERVICE$',regex=True) |
    dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('^ROOM SERVICE$',regex=True)
]


print(f"Violations found: {len(ic14_violations)}")
dataset[MENU].loc[ic14_violations.index][['event','occasion']]


Violations found: 26


,event,occasion
447,WINE LIST,NaN
494,WINE LIST,DAILY
584,WINE LIST,NaN
823,WINE LIST,DAILY
886,WINE LIST,NaN
1253,WINE LIST,DAILY
5674,WINE LIST FOR COMPOSITE CAR SERVICE,NaN
6237,WINE LIST,PRICED WINE LIST
6454,DAILY PRICE LIST,NaN
7201,WINE LIST,NaN


In [ ]:
# IC 14: event and occasion field contains pure information on menu. Cleaning
# e.g WINE XXX LIST/ROOM SERVICE
# those are not helpful to identify what event there is 

event_toclean_filter = dataset[MENU]['event'].fillna('na').str.upper().str.contains('[A-z]+\sLIST',regex=True) | dataset[MENU]['event'].fillna('na').str.upper().str.contains('^ROOM SERVICE$',regex=True) 
dataset[MENU].loc[event_toclean_filter,'event']=np.nan

occasion_toclean_filter = dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('[A-z]+\sLIST',regex=True) | dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('^ROOM SERVICE$',regex=True)
dataset[MENU].loc[occasion_toclean_filter,'occasion']=np.nan


ic14_violations_after = dataset[MENU][
    dataset[MENU]['event'].fillna('na').str.upper().str.contains('[A-z]+\sLIST',regex=True) | 
    dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('[A-z]+\sLIST',regex=True) |
    dataset[MENU]['event'].fillna('na').str.upper().str.contains('^ROOM SERVICE$',regex=True) |
    dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('^ROOM SERVICE$',regex=True)
]

print(f"IC 14 Violations After Cleaning: {len(ic14_violations_after)}")
print(f"Dish dataset size: {dataset[MENU].shape}")
dataset[MENU].loc[ic14_violations_after.index,['occasion','event']].head()


IC 14 Violations After Cleaning: 0
Dish dataset size: (17545, 22)


,occasion,event


In [ ]:
# IC 15: A DAILY MENU that was NOT used for a special occastion/holiday. It should be exlcuded from our analysis.

ic15_violations = dataset[MENU][
    (dataset[MENU]['event'].fillna('na').str.upper().str.contains('DAILY|REGULAR') | 
     dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('DAILY|REGULAR') ) &
    (~dataset[MENU]['event'].fillna('na').str.contains('HOLIDAY|FOR|TO|OF')) & 
    (~dataset[MENU]['occasion'].fillna('na').str.contains('HOLIDAY|FOR|TO|OF'))
]


print(f"Violations found: {len(ic15_violations)}")
dataset[MENU].loc[ic15_violations.index][['event','occasion']]


Violations found: 1886


,event,occasion
37,BREAKFAST,DAILY
38,LUNCH,DAILY
41,BREAKFAST,DAILY
42,LUNCH DINNER,DAILY
43,SUPPER,DAILY
...,...,...
9982,DINNER,DAILY MENU
9983,DINNER,DAILY MENU
9984,BREAKFAST,DAILY MENU
10088,DAILY MENU AND RATES,NaN


In [ ]:
# IC 15: A DAILY MENU that was NOT used for a special occastion/holiday. It should be exlcuded from our analysis. Cleaning

## Remove those daily events from occastion_event column
print(f"Before Cleaning Applied Dish dataset size: {dataset[MENU].shape}")


## Create a new column special_occasion with both occation and event information combined
## Use Occasioin first, if occasion is NaN use event. Because event if more noisy. 
dataset[MENU]['special_occasion'] = np.where( dataset[MENU]['occasion'].isna(), dataset[MENU]['event'], dataset[MENU]['occasion'])

dataset[MENU]['special_occasion'] = dataset[MENU]['special_occasion'].fillna("").str.replace("BREAKFAST|DINNER|SUPPER|LUNCHEON|LUNCH|TIFFIN","", regex=True)

## below is some trivial cleaning. removal of MENU, A LA CARTE, OR, & /

dataset[MENU]['special_occasion'] = dataset[MENU]['special_occasion'].str.replace("MENU|MEAL|(A LA CARTE)","",regex=True)
dataset[MENU]['special_occasion'] = dataset[MENU]['special_occasion'].str.replace(" OR ","")
dataset[MENU]['special_occasion'] = dataset[MENU]['special_occasion'].str.replace("&","")
dataset[MENU]['special_occasion'] = dataset[MENU]['special_occasion'].str.replace("/","")
dataset[MENU]['special_occasion'] = dataset[MENU]['special_occasion'].str.replace("-"," ")
dataset[MENU]['special_occasion'] = dataset[MENU]['special_occasion'].str.replace("^AND$","",regex=True)
dataset[MENU]['special_occasion'] = dataset[MENU]['special_occasion'].str.strip()


ic15_violations_withOR = dataset[MENU][
    (dataset[MENU]['event'].fillna('na').str.upper().str.contains('DAILY|REGULAR') | 
     dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('DAILY|REGULAR') ) &
    (~dataset[MENU]['event'].fillna('na').str.contains('HOLIDAY|FOR|TO|OF')) & 
    (~dataset[MENU]['occasion'].fillna('na').str.contains('HOLIDAY|FOR|TO|OF'))
]

dataset[MENU].loc[ ic15_violations_withOR.index, 'special_occasion' ] = ""

ic15_violations_after = dataset[MENU][
    (dataset[MENU]['special_occasion'].fillna('na').str.upper().str.contains('DAILY|REGULAR') & 
    (~dataset[MENU]['special_occasion'].fillna('na').str.contains('HOLIDAY|FOR|TO|OF')))
]

print(f"IC 15 Violations After Cleaning: {len(ic15_violations_after)}")
print(f"Dish dataset size: {dataset[MENU].shape}")
dataset[MENU].loc[ic15_violations.index,['special_occasion','occasion','event']].head()

Before Cleaning Applied Dish dataset size: (17545, 22)
IC 15 Violations After Cleaning: 0
Dish dataset size: (17545, 23)


,special_occasion,occasion,event
37,,DAILY,BREAKFAST
38,,DAILY,LUNCH
41,,DAILY,BREAKFAST
42,,DAILY,LUNCH DINNER
43,,DAILY,SUPPER


In [ ]:
# IC 16: A WEEKDAY MENU that was NOT used for a special occastion/holiday. It should be exlcuded from our analysis.

ic16_violations = dataset[MENU][ 
    (dataset[MENU]['event'].fillna('na').str.upper().str.contains('MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY') |
     dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY')) & 
    (~dataset[MENU]['event'].fillna('na').str.upper().str.contains('THANKSGIVING|CHRISTMAS|EASTER|OF|TO|FOR')) & 
    (~dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('THANKSGIVING|CHRISTMAS|EASTER|OF|TO|FOR'))
]


print(f"Violations found: {len(ic16_violations)}")
dataset[MENU].loc[ic16_violations.index][['event','occasion']]

Violations found: 41


,event,occasion
34,SUNDAY DINNER,NaN
825,SUNDAY DINNER,DAILY
893,SUNDAY DINNER,NaN
1019,SUNDAY DINNER,NaN
1022,SUNDAY SUPPER,NaN
1212,SUNDAY DINNER,NaN
1282,SUNDAY TIFFIN,NaN
2509,SUNDAY DINNER,NaN
4399,SUNDAY DINNER,NaN
4630,SUNDAY DINNER,DAILY MEAL


In [ ]:
# IC 16: A WEEKDAY MENU that was NOT used for a special occastion/holiday. It should be exlcuded from our analysis. Cleaning

## Remove those weekday menus events from occastion_event column
print(f"Before Cleaning Applied Dish dataset size: {dataset[MENU].shape}")

ic16_violations_withOR = dataset[MENU][ 
    (dataset[MENU]['event'].fillna('na').str.upper().str.contains('MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY') |
     dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY')) & 
    (~dataset[MENU]['event'].fillna('na').str.upper().str.contains('THANKSGIVING|CHRISTMAS|EASTER|OF|TO|FOR')) & 
    (~dataset[MENU]['occasion'].fillna('na').str.upper().str.contains('THANKSGIVING|CHRISTMAS|EASTER|OF|TO|FOR'))
]


dataset[MENU].loc[ ic16_violations_withOR.index, 'special_occasion' ] = ""

ic16_violations_after = dataset[MENU][
    (dataset[MENU]['special_occasion'].fillna('na').str.upper().str.contains('MONDAY|TUESDAY|WEDNESDAY|THURSDAY|FRIDAY|SATURDAY|SUNDAY') & 
    (~dataset[MENU]['special_occasion'].fillna('na').str.contains('THANKSGIVING|CHRISTMAS|EASTER|OF|TO|FOR')))
]

print(f"IC 16 Violations After Cleaning: {len(ic16_violations_after)}")
print(f"Dish dataset size: {dataset[MENU].shape}")
dataset[MENU].loc[ic16_violations.index,['special_occasion','occasion','event']].head() 

Before Cleaning Applied Dish dataset size: (17545, 23)
IC 16 Violations After Cleaning: 0
Dish dataset size: (17545, 23)


,special_occasion,occasion,event
34,,NaN,SUNDAY DINNER
825,,DAILY,SUNDAY DINNER
893,,NaN,SUNDAY DINNER
1019,,NaN,SUNDAY DINNER
1022,,NaN,SUNDAY SUPPER


In [ ]:
dataset[MENU].to_csv(Path(OUTPUT_FOLDER) / ('Menu_fixed_clean_occasion.csv'), index=False)